In [1]:
# Import libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models, utils
from gensim.models import LdaModel
from gensim import similarities
from gensim.models.phrases import Phrases, Phraser

import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords, wordnet
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from cleantext import clean

from wordcloud import WordCloud
import seaborn as sns

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
#Import data files
jd_ba = pd.read_csv("../data/jd_ba.csv",encoding='utf-8')
jd_ba.head()

,description
0,Company Overview At Memorial Sloan Kettering M...
1,We are seeking for an energetic and collaborat...
2,For more than a decade Asembia has been workin...
3,Job Description Summary The Information Securi...
4,Magnite is the world s largest independent sel...


### Training LDA model using words

In [3]:
def preprocess_text(text):
    # Tokenize text
    tokens = text.lower().split()
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens

In [4]:
# Preprocess job descriptions
preprocessed_job_descriptions = jd_ba['description'].apply(preprocess_text)

In [5]:
# Create dictionary representation of job descriptions
dictionary = corpora.Dictionary(preprocessed_job_descriptions)

In [6]:
# Create document-term matrix
doc_term_matrix = [dictionary.doc2bow(desc) for desc in preprocessed_job_descriptions]

In [7]:
# Train LDA model
lda_model = models.LdaModel(doc_term_matrix, num_topics=10, id2word=dictionary, passes=10)

In [8]:
# Extract and print top topics
top_topics = lda_model.show_topics(num_topics=10, num_words=10)

In [9]:
for topic in top_topics:
    print(topic)

(0, '0.026*"0" + 0.023*"9" + 0.021*"service" + 0.021*"florida" + 0.019*"5" + 0.018*"university" + 0.016*"group" + 0.014*"3" + 0.014*"financial" + 0.013*"llc"')
(1, '0.011*"team" + 0.010*"work" + 0.009*"business" + 0.009*"company" + 0.008*"customer" + 0.007*"opportunity" + 0.006*"experience" + 0.006*"product" + 0.006*"client" + 0.006*"service"')
(2, '0.200*"3" + 0.045*"4" + 0.042*"6" + 0.039*"7" + 0.025*"citi" + 0.022*"1" + 0.021*"5" + 0.017*"8" + 0.012*"technology" + 0.012*"staffing"')
(3, '0.016*"experience" + 0.011*"must" + 0.010*"position" + 0.010*"program" + 0.010*"year" + 0.010*"required" + 0.010*"may" + 0.009*"financial" + 0.009*"management" + 0.008*"work"')
(4, '0.011*"status" + 0.009*"employment" + 0.009*"system" + 0.009*"information" + 0.009*"experience" + 0.008*"disability" + 0.007*"data" + 0.007*"support" + 0.007*"applicant" + 0.006*"opportunity"')
(5, '0.034*"business" + 0.017*"requirement" + 0.017*"experience" + 0.015*"system" + 0.014*"process" + 0.014*"project" + 0.011*"t

In [10]:
topic_labels = []
num_topics = 10
for topic_id in range(num_topics):
    top_words = lda_model.show_topic(topic_id, topn=10)  # Get the top 10 words for each topic
    words = [word for word, _ in top_words]
    topic_label = ', '.join(words)  # Join the words into a single string
    topic_labels.append(topic_label)

In [11]:
# Print the topic labels
for i, label in enumerate(topic_labels):
    print(f"Topic {i}: {label}")

Topic 0: 0, 9, service, florida, 5, university, group, 3, financial, llc
Topic 1: team, work, business, company, customer, opportunity, experience, product, client, service
Topic 2: 3, 4, 6, 7, citi, 1, 5, 8, technology, staffing
Topic 3: experience, must, position, program, year, required, may, financial, management, work
Topic 4: status, employment, system, information, experience, disability, data, support, applicant, opportunity
Topic 5: business, requirement, experience, system, process, project, team, work, skill, management
Topic 6: salesforce, cloud, experience, solution, business, system, service, team, crm, project
Topic 7: data, business, experience, analysis, skill, analyst, team, year, analytics, work
Topic 8: security, service, federal, technology, experience, work, operation, enture, job, level
Topic 9: credit, bank, loan, union, banking, financial, risk, aml, mso, vystar


### Training LDA model using bigrams

In [12]:
def preprocess_text_bigram(text):
    # Tokenize text into sentences
    sentences = [sent for sent in nltk.sent_tokenize(text)]
    
    # Tokenize sentences into words
    tokenized_sentences = [utils.simple_preprocess(sent) for sent in sentences]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    sentences = [[token for token in sent if token not in stop_words] for sent in tokenized_sentences]
    
    # Create bigrams
    bigram_model = Phrases(sentences, min_count=5, threshold=100)
    bigram_phraser = Phraser(bigram_model)
    bigram_sentences = [bigram_phraser[sent] for sent in sentences]
    
    # Convert bigrams into single words using '_'
    for i in range(len(bigram_sentences)):
        for j in range(len(bigram_sentences[i])):
            if '_' in bigram_sentences[i][j]:
                bigram_sentences[i][j] = bigram_sentences[i][j].replace('_', '')
    
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    preprocessed_sentences = [[lemmatizer.lemmatize(token) for token in sent] for sent in bigram_sentences]
    
    # Flatten sentences into a single list of words
    words = [word for sent in preprocessed_sentences for word in sent]
    
    return words

In [13]:
# Preprocess job descriptions
preprocessed_job_descriptions_bigram = jd_ba['description'].apply(preprocess_text_bigram)

In [14]:
# Identify bigrams and add them to the dictionary
bigram_model = Phrases(preprocessed_job_descriptions_bigram, min_count=5, threshold=100)
bigram_phraser = Phraser(bigram_model)

for i in range(len(preprocessed_job_descriptions_bigram)):
    for token in bigram_phraser[preprocessed_job_descriptions_bigram[i]]:
        if '_' in token:
            preprocessed_job_descriptions_bigram[i].append(token)

In [15]:
# Create dictionary representation of job descriptions
dictionary_bigram = corpora.Dictionary(preprocessed_job_descriptions_bigram.values)

In [16]:
# Create document-term matrix
doc_term_matrix_bigram = [dictionary.doc2bow(desc) for desc in preprocessed_job_descriptions_bigram]

In [17]:
# Train LDA model
lda_model_bigram = models.LdaModel(doc_term_matrix_bigram, num_topics=10, id2word=dictionary, passes=10)

In [18]:
# Extract and print top topics
top_topics_bigram = lda_model.show_topics(num_topics=10, num_words=10)

for topic in top_topics_bigram:
    print(topic)# Extract and print top topics

(0, '0.026*"0" + 0.023*"9" + 0.021*"service" + 0.021*"florida" + 0.019*"5" + 0.018*"university" + 0.016*"group" + 0.014*"3" + 0.014*"financial" + 0.013*"llc"')
(1, '0.011*"team" + 0.010*"work" + 0.009*"business" + 0.009*"company" + 0.008*"customer" + 0.007*"opportunity" + 0.006*"experience" + 0.006*"product" + 0.006*"client" + 0.006*"service"')
(2, '0.200*"3" + 0.045*"4" + 0.042*"6" + 0.039*"7" + 0.025*"citi" + 0.022*"1" + 0.021*"5" + 0.017*"8" + 0.012*"technology" + 0.012*"staffing"')
(3, '0.016*"experience" + 0.011*"must" + 0.010*"position" + 0.010*"program" + 0.010*"year" + 0.010*"required" + 0.010*"may" + 0.009*"financial" + 0.009*"management" + 0.008*"work"')
(4, '0.011*"status" + 0.009*"employment" + 0.009*"system" + 0.009*"information" + 0.009*"experience" + 0.008*"disability" + 0.007*"data" + 0.007*"support" + 0.007*"applicant" + 0.006*"opportunity"')
(5, '0.034*"business" + 0.017*"requirement" + 0.017*"experience" + 0.015*"system" + 0.014*"process" + 0.014*"project" + 0.011*"t

In [19]:
# Save the trained LDA model
lda_model.save("../model/lda_model_ba")
lda_model_bigram.save("../model/lda_model_ba_bigram")